In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
import gspread
from google.oauth2 import service_account
import json
import google.auth
from google.auth.transport.requests import AuthorizedSession
from google.oauth2.credentials import Credentials
from oauth2client.service_account import ServiceAccountCredentials



#
#
# RUN SELENIUM

In [2]:
driver = webdriver.Chrome(r"/Users/jarkrunglerdkriangkrai/Github/setscraping/chromedriver")

#
#
# GET DATA

In [3]:
driver.get('http://siamchart.com/stock/')

In [4]:
data = driver.page_source

#
#
# CONVERT TO DATAFRAME

In [5]:
pd.read_html(data)[2]

,Name,No.,Links,Sign,Last,Chg%,Volume,Value (k),MCap (M),P/E,...,ROA%,ROE%,NPM%,Yield%,FFloat%,MG%,Magic1,Magic2,PEG,CG
0,Average,NaN,NaN,NaN,19.75,1.12,5809862.79,77163.02,25663.1,35.63,...,5.52,7.16,12.08,0.10,41.38,NaN,616.83,614.25,2.00,NaN
1,24CS,1.0,i,NaN,2.90,-1.36,643200.00,1871.00,0.0,NaN,...,NaN,NaN,NaN,-,30.23,NaN,NaN,NaN,NaN,NaN
2,2S,2.0,i | 1 | 2 | 3,NaN,3.28,1.86,87900.00,286.00,1804.0,NaN,...,-2.64,-4.19,-2.68,13.27,57.50,NaN,NaN,NaN,NaN,NaN
3,3K-BAT,3.0,i | 1 | 3,NaN,62.25,0.00,0.00,0.00,4880.0,15.76,...,10.38,13.01,5.35,0.50,2.10,NaN,483.00,452.00,0.16,NaN
4,7UP,4.0,i | 1 | 2 | 3,NaN,0.80,1.27,10040800.00,8034.00,4114.0,121.48,...,0.31,1.32,-12.96,NaN,63.37,NaN,1205.00,1260.00,-5.02,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,YGG,819.0,i | 1 | 3,NaN,8.05,1.90,1386300.00,11080.00,4846.0,39.83,...,23.60,24.66,36.59,0.89,45.15,NaN,557.00,508.00,0.49,NaN
820,YONG,820.0,i,NaN,2.14,4.90,11671800.00,24674.00,1455.0,16.43,...,9.49,11.42,9.25,NaN,27.24,NaN,536.00,484.00,NaN,NaN
821,YUASA,821.0,i | 1 | 3,NaN,15.50,-3.13,1866000.00,30178.00,1668.0,15.25,...,8.95,11.03,4.15,2.16,15.27,NaN,526.00,484.00,-25.33,NaN
822,ZEN,822.0,i | 1 | 3,NaN,17.00,1.19,225200.00,3825.00,5100.0,47.00,...,7.31,9.29,4.02,NaN,27.34,NaN,851.00,805.00,-2.73,NaN


In [6]:
df = pd.read_html(data)[2].drop(['No.','Links','CG','Sign','MG%'],axis=1).tail(-1).set_index('Name')
df

,Last,Chg%,Volume,Value (k),MCap (M),P/E,P/BV,D/E,DPS,EPS,ROA%,ROE%,NPM%,Yield%,FFloat%,Magic1,Magic2,PEG
Name,,,,,,,,,,,,,,,,,,
24CS,2.90,-1.36,643200.0,1871.0,0.0,NaN,-,NaN,-,0.00,NaN,NaN,NaN,-,30.23,NaN,NaN,NaN
2S,3.28,1.86,87900.0,286.0,1804.0,NaN,0.90,0.26,0.27,0.91,-2.64,-4.19,-2.68,13.27,57.50,NaN,NaN,NaN
3K-BAT,62.25,0.00,0.0,0.0,4880.0,15.76,2.05,0.65,0.06,0.54,10.38,13.01,5.35,0.50,2.10,483.0,452.0,0.16
7UP,0.80,1.27,10040800.0,8034.0,4114.0,121.48,1.63,0.38,NaN,0.04,0.31,1.32,-12.96,NaN,63.37,1205.0,1260.0,-5.02
A,5.05,0.00,3700.0,19.0,4949.0,NaN,1.56,3.42,NaN,0.20,2.11,-2.60,2.87,NaN,24.92,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YGG,8.05,1.90,1386300.0,11080.0,4846.0,39.83,7.97,0.05,0.08,0.20,23.60,24.66,36.59,0.89,45.15,557.0,508.0,0.49
YONG,2.14,4.90,11671800.0,24674.0,1455.0,16.43,1.81,0.63,NaN,0.13,9.49,11.42,9.25,NaN,27.24,536.0,484.0,NaN
YUASA,15.50,-3.13,1866000.0,30178.0,1668.0,15.25,1.63,0.59,0.35,1.02,8.95,11.03,4.15,2.16,15.27,526.0,484.0,-25.33


#
#
# UPDATE TO GOOGLE SHEETS

In [7]:
scope = ['https://docs.google.com/spreadsheets/d/1sDLJJubzeMhc_U4xUe-P3rvSF7t7pV8tRKeabtux6WQ/edit#gid=0']
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/jarkrunglerdkriangkrai/Github/setscraping/client_secret_92929305327-1dct19i137cgadggog4vdltkb7r1m2pb.apps.googleusercontent.com.json', scope)
client = gspread.authorize(creds)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jarkrunglerdkriangkrai/Github/setscraping/client_secret_92929305327-1dct19i137cgadggog4vdltkb7r1m2pb.apps.googleusercontent.com.json'

In [ ]:
#creds, project = google.auth.default(scopes=['https://www.googleapis.com/auth/spreadsheets'])
#service = build('sheets', 'v4', credentials=creds)

In [ ]:
sheet_name = 'Sheet1'
worksheet_name = 'set scraping'
sheet = client.open(sheet_name).worksheet(worksheet_name)

In [ ]:
sheet.update('A1',df)